## Pykrx를 사용한 금융 데이터 수집 PART 2


In [6]:
import pandas as pd
from pykrx import stock
from pykrx import bond
from time import sleep
 
from datetime import datetime
import os
import time

## 날짜

In [11]:
now = datetime.now()
today_date1 = now.strftime('%Y%m%d')

today_date1 = '20230701'
start_date = '20180101'
start_date = '20230601'

## 인덱스 리스트 생성

In [17]:
kor_index_list_df = pd.DataFrame()
market_list = ['KOSPI', 'KOSDAQ'] 
 
for market_nm in market_list:
    kor_index_list = stock.get_index_ticker_list(market=market_nm)
    for index_tickers in kor_index_list:
        index_name = stock.get_index_ticker_name(index_tickers)
        df = pd.DataFrame({'kospi_index':index_tickers,
                           'kospi_index_nm':index_name,
                           'market': market_nm
                          }, index = [0])
        kor_index_list_df = pd.concat([kor_index_list_df,df])
        
kor_index_list_df = kor_index_list_df.reset_index(drop = True)
kor_index_list_df.head()

,index_code,index_code_nm,market
0,1001,코스피,KOSPI
1,1002,코스피 대형주,KOSPI
2,1003,코스피 중형주,KOSPI
3,1004,코스피 소형주,KOSPI
4,1005,음식료품,KOSPI


In [19]:
kor_index_ticker_list = kor_index_list_df['index_code']
kor_index_list_df.to_csv(f'data_crawler/kor_kospi_index_list_df.csv', index=False, mode='w')

In [20]:
kor_index_ticker_list

0     1001
1     1002
2     1003
3     1004
4     1005
      ... 
92    2214
93    2215
94    2216
95    2217
96    2218
Name: index_code, Length: 97, dtype: object

### 인덱스 OHLCV 조회

In [27]:
for index_ticker in kor_index_ticker_list:
    file_name = 'kor_index_ohlcv'

    try:
        df_raw = stock.get_index_ohlcv(start_date, today_date1, index_ticker)
        df_raw = df_raw.reset_index()
        df_raw['index_ticker'] = index_ticker
        
        if not os.path.exists(f'data_crawler/{file_name}.csv'):
            df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='w')
        else:
            df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='a', header=False)
        
        print('success')    
    except:
        print('fail')

success
success


In [28]:
df_raw.head(3).T

,0,1,2
코스피 대형주,,,
날짜,2023-06-01 00:00:00,2023-06-02 00:00:00,2023-06-05 00:00:00
시가,2574.46,2588.92,2621.76
고가,2582.46,2604.71,2621.76
저가,2566.04,2586.47,2608.74
종가,2570.3,2604.71,2616.49
거래량,76234358,73355328,83302457
거래대금,5240317905443,5477095023315,5074544647627
상장시가총액,1633027742856730,1654886981280910,1662376324326780
index_ticker,1002,1002,1002


### 인덱스 등락률

In [31]:
for index_ticker in kor_index_ticker_list[:2]:
    file_name = 'kor_index_ticker_fundamental'
    
    try:
        df_raw = stock.get_index_fundamental(start_date, today_date1, index_ticker)
        df_raw = df_raw.reset_index()
        df_raw['index_ticker'] = index_ticker
        
        if not os.path.exists(f'data_crawler/{file_name}.csv'):
            df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='w')
        else:
            df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='a', header=False)
        
        print('success')    
    except:
        print('fail')

success
success


In [33]:

df_raw.head(3).T


,0,1,2
날짜,2023-06-01 00:00:00,2023-06-02 00:00:00,2023-06-05 00:00:00
종가,2570.3,2604.71,2616.49
등락률,-0.4,1.34,0.45
PER,13.92,14.11,14.17
선행PER,0.0,0.0,0.0
PBR,1.06,1.07,1.08
배당수익률,1.9,1.88,1.87
index_ticker,1002,1002,1002


In [34]:
kor_index_list_df

,index_code,index_code_nm,market
0,1001,코스피,KOSPI
1,1002,코스피 대형주,KOSPI
2,1003,코스피 중형주,KOSPI
3,1004,코스피 소형주,KOSPI
4,1005,음식료품,KOSPI
...,...,...,...
92,2214,코스닥 150 필수소비재,KOSDAQ
93,2215,코스닥 150 자유소비재,KOSDAQ
94,2216,코스닥 150 정보기술,KOSDAQ
95,2217,코스닥 150 헬스케어,KOSDAQ


### 인덱스 구성종목 조회

In [52]:
index_ticker_info = pd.DataFrame()
for i in kor_index_list_df['index_code']:
    pdf = stock.get_index_portfolio_deposit_file(str(i))
    df = pd.DataFrame({'ticker':pdf,
                       'index_code': str(i)})
    index_ticker_info = pd.concat([index_ticker_info, df])
index_ticker_info = index_ticker_info.reset_index(drop = True)

In [53]:
index_ticker_info.head()

,ticker,index_code
0,005930,1001
1,373220,1001
2,000660,1001
3,207940,1001
4,006400,1001


In [54]:
kor_index_list_df.head()

,index_code,index_code_nm,market
0,1001,코스피,KOSPI
1,1002,코스피 대형주,KOSPI
2,1003,코스피 중형주,KOSPI
3,1004,코스피 소형주,KOSPI
4,1005,음식료품,KOSPI


In [60]:
index_ticker_info_2  = pd.merge(index_ticker_info, kor_index_list_df,
        how = 'left',
        on = 'index_code')

In [62]:
index_ticker_info_2.head()

,ticker,index_code,index_code_nm,market
0,005930,1001,코스피,KOSPI
1,373220,1001,코스피,KOSPI
2,000660,1001,코스피,KOSPI
3,207940,1001,코스피,KOSPI
4,006400,1001,코스피,KOSPI


In [63]:
ticker_list_df.head()

,ticker,corp_name,market
0,095570,AJ네트웍스,KOSPI
1,006840,AK홀딩스,KOSPI
2,027410,BGF,KOSPI
3,282330,BGF리테일,KOSPI
4,138930,BNK금융지주,KOSPI


In [66]:
index_ticker_master  = pd.merge(index_ticker_info_2, ticker_list_df[['ticker','corp_name']],
        how = 'left',
        on = 'ticker')

In [68]:
index_ticker_master.head()

,ticker,index_code,index_code_nm,market,corp_name
0,005930,1001,코스피,KOSPI,삼성전자
1,373220,1001,코스피,KOSPI,LG에너지솔루션
2,000660,1001,코스피,KOSPI,SK하이닉스
3,207940,1001,코스피,KOSPI,삼성바이오로직스
4,006400,1001,코스피,KOSPI,삼성SDI
